<a href="https://colab.research.google.com/github/LuizScarparo/DataScience/blob/main/Dashboard_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1) Crie o seu report.md
%%bash
cat << 'EOF' > report.md
# Relatório do Projeto de Análise de Estatísticas de Jogadores

## 1. Tema do Projeto
Análise exploratória de estatísticas de jogadores de futebol usando Python (pandas, matplotlib, plotly) e construção de um dashboard interativo com Dash.

## 2. Repositório no GitHub
https://github.com/seu-usuario/seu-repo-toca-ficha-dashboard

## 3. Dataset Utilizado
- **Origem:** arquivo `database.csv`
- **Variáveis principais:** Jogador, Time, Nação, Pos., Min., Gols, Assis., Cmp%, …
- **Transformações:**
  1. Conversão de strings numéricas (remoção de “%”, vírgula → ponto).
  2. Extração de anos de “26-150” → coluna `Anos`.
  3. Cálculo de `Gols_per90 = Gols/(Min./90)`.

## 4. Modelos / Métodos
- Regressão linear (numpy.polyfit) em Min.×Gols_per90 e Anos×CrtsA
- Dashboard em Dash com callbacks interativos

## 5. Resultados
- Top‐10 artilheiros (barra horizontal)
- Correlação Idade vs. Cartões Amarelos: r ≈ 0.02 (fraca)
- Dashboards interativos para explorar nacionalidades, posições e heatmap de correlação.

EOF


In [2]:
# 2) Instale Pandoc e TeXLive (pode levar alguns minutos)
!apt-get update -qq
!apt-get install -qq pandoc texlive-xetex

# 3) Converta o Markdown em PDF
!pandoc report.md \
  --from markdown+yaml_metadata_block \
  --pdf-engine=xelatex \
  -V geometry:margin=1in \
  -o report.pdf

# 4) Faça o download do PDF gerado
from google.colab import files
files.download('report.pdf')


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1build1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1build1) ...
Selecting previously unselected package fonts-lato.
Preparing to unpack .../01-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../02-poppler-data_0.4.11-1_all.deb ...
Unpacking poppler-data (0.4.11-1) ...
Selecting previously unselected package tex-common.
Preparing to unpack .../03-tex-common_6.17_all.deb ...
Unpacking tex-common (6.17) ...
Selecting previously unselect

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
# instale o Pandoc e um engine LaTeX mínimo
!apt-get update -qq
!apt-get install -qq pandoc texlive-xetex


^C
^C


In [29]:
# === Cell 2: Geração dos Gráficos e Salvamento em PNG (corrigido) ===
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 2.1 Carrega o dataset (faça upload antes em /content/database.csv)
df = pd.read_csv('/content/database.csv', sep=',', encoding='utf-8', dtype={'Idade': str})

# 2.2 Pré-processamento de 'Idade'
# usa argumentos nomeados para split e converte com to_numeric
df['Anos'] = (
    df['Idade']
      .astype(str)
      .str.split(pat='-', n=1).str[0]         # divide em duas partes, pega a primeira
)
df['Anos'] = pd.to_numeric(df['Anos'], errors='coerce')

# converte CrtsA e Cmp%
df['CrtsA'] = pd.to_numeric(df['CrtsA'], errors='coerce')
df['Cmp%']  = pd.to_numeric(df['Cmp%'].str.replace(',','.'), errors='coerce')

# calcula Gols_per90
df['Gols_per90'] = df['Gols'] / (df['Min.'] / 90)

# 2.3 Distribuição de Nacionalidades
counts = df['Nação'].value_counts()
plt.figure(figsize=(6,6))
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Distribuição de Nacionalidades')
plt.savefig('nacionalidades.png', bbox_inches='tight')
plt.close()

# 2.4 Distribuição de Posições
counts = df['Pos.'].value_counts()
perc   = counts / counts.sum() * 100
major  = counts[perc >= 2].copy()
others = counts[perc < 2].sum()
if others > 0:
    major['Outros'] = others
plt.figure(figsize=(6,6))
plt.pie(major, labels=major.index, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Distribuição de Posições')
plt.savefig('posicoes.png', bbox_inches='tight')
plt.close()

# 2.7 Idade vs. Cartões Amarelos
data_ic = df.dropna(subset=['Anos','CrtsA'])
m_ic, b_ic = np.polyfit(data_ic['Anos'], data_ic['CrtsA'], 1)
plt.figure(figsize=(6,4))
plt.scatter(data_ic['Anos'], data_ic['CrtsA'], alpha=0.6, s=30)
plt.plot(data_ic['Anos'], m_ic*data_ic['Anos']+b_ic, '--', lw=2, color='red')
plt.title('Idade vs. Cartões Amarelos')
plt.xlabel('Anos'); plt.ylabel('Cartões Amarelos')
plt.savefig('idade_cartoes.png', bbox_inches='tight')
plt.close()

# 2.8 Top-10 Artilheiros
top10 = df.groupby('Jogador')['Gols'].sum().nlargest(10)
plt.figure(figsize=(8,6))
(top10[::-1]).plot(kind='barh')
plt.title('Top 10 Artilheiros')
plt.xlabel('Total de Gols'); plt.ylabel('')
plt.savefig('top10.png', bbox_inches='tight')
plt.close()

# 2.9 Heatmap de Correlação
cols = ['Gols','Assis.','xG','xAG','Cmp%','PrgP','Conduções']
corr = df[cols].dropna().corr()
plt.figure(figsize=(6,6))
plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar()
plt.xticks(range(len(cols)), cols, rotation=45, ha='right')
plt.yticks(range(len(cols)), cols)
plt.title('Heatmap de Correlação')
plt.savefig('heatmap.png', bbox_inches='tight')
plt.close()

# 2.10 Eficiência de Gols por 90 Minutos
data_eff = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Min.','Gols_per90'])
m_eff, b_eff = np.polyfit(data_eff['Min.'], data_eff['Gols_per90'], 1)
plt.figure(figsize=(6,4))
plt.scatter(data_eff['Min.'], data_eff['Gols_per90'], alpha=0.6, s=30)
plt.plot([0, data_eff['Min.'].max()],
         [b_eff, m_eff*data_eff['Min.'].max()+b_eff],
         '--', lw=2, color='green')
plt.title('Gols/90 min vs Minutos Jogados')
plt.xlabel('Minutos Jogados'); plt.ylabel('Gols/90 min')
plt.savefig('eficiencia.png', bbox_inches='tight')
plt.close()


In [33]:
# Cell 3: Montagem Dinâmica do report.md (com Resumo dos Resultados)

with open('report.md','w') as f:
    f.write("# Relatório Completo de Estatísticas de Jogadores\n\n")
    f.write("## 1. Tema do Projeto\n")
    f.write("Análise exploratória de estatísticas de futebol usando Python e Dash.\n\n")
    f.write("## 2. Repositório no GitHub\n")
    f.write("https://github.com/LuizScarparo/DataScience.git\n\n")
    f.write("## 3. Dataset Utilizado\n")
    f.write("- **Origem:** `/content/database.csv`\n")
    f.write("- **Variáveis principais:** Jogador, Time, Nação, Pos., Min., Gols, Assis., Cmp%, xG, xAG\n")
    f.write("- **Resumo da base de dados:**\n")
    f.write("- Dataset contendo informações sobre o campeonato brasileiro de 2015\n")
    f.write("- **Fonte dos dados: ** https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024/discussion?sort=undefined\n")
    f.write("## 4. Modelos e Métodos\n")
    f.write("- Regressão Linear (numpy.polyfit)\n")
    f.write("- Visualizações: pie, bar, scatter, heatmap\n\n")
    f.write("## 5. Resultados Obtidos\n")
    f.write("### 5.1 Top 10 Artilheiros\n")

with open('report.md','a') as f:
    for title,img in sections:
        f.write(f"### {title}\n\n")
        f.write(f"![{title}]({img})\n\n")

    # ===== RESUMO DOS RESULTADOS =====
    f.write("## 6. Resumo dos Resultados\n")
    f.write("Em síntese, os gráficos revelaram que a grande maioria dos jogadores é de nacionalidade brasileira, seguida por um pequeno contingente de argentinos e europeus, com as demais nacionalidades somando menos de 5% cada (agrupadas em “Outros”). Na distribuição de posições, defensores e meio-campistas respondem por cerca de 70% do total, enquanto alas, atacantes e goleiros compõem o restante (com posições abaixo de 2% também agrupadas em “Outros”). Os boxplots mostraram que meio-campistas apresentam a maior mediana de assistências, ao passo que atacantes lideram na mediana de gols, confirmando sua função ofensiva. A análise de idade versus cartões amarelos apontou uma correlação de Pearson próxima a r = 0,02, indicando que a idade pouco influencia o número de advertências. No gráfico de eficiência (gols por 90 minutos), nota-se uma leve tendência positiva em relação aos minutos jogados, sinal de que jogadores mais utilizados mantêm eficiência de finalização estável. Por fim, o heatmap de correlação destacou forte associação entre gols e xG, e entre assistências e xAG, validando o poder preditivo dessas métricas avançadas.\n\n")

    # ===== CONCLUSÕES =====
    f.write("## 7. Conclusões\n")
    f.write("Este relatório apresentou uma visão completa dos dados: top scorers, distribuições e correlações de desempenho.\n\n\n\n")

# lista dinâmica de artilheiros
df = pd.read_csv('/content/database.csv', sep=',', encoding='utf-8', dtype={'Idade': str})
top10 = df.groupby('Jogador')['Gols'].sum().nlargest(10)
with open('report.md','a') as f:
    for i,(jog,gols) in enumerate(top10.items(),1):
        f.write(f"{i}. **{jog}**: {gols} gols\n")
    f.write("\n")

# inclusão das figuras
sections = [
    ("Distribuição de Nacionalidades","nacionalidades.png"),
    ("Distribuição de Posições","posicoes.png"),
    ("Idade vs. Cartões Amarelos","idade_cartoes.png"),
    ("Top 10 Artilheiros","top10.png"),
    ("Heatmap de Correlação","heatmap.png"),
    ("Eficiência de Gols por 90 Minutos","eficiencia.png")
]



In [34]:
# converte o Markdown em PDF
!pandoc report.md \
  --from markdown+yaml_metadata_block \
  --pdf-engine=xelatex \
  -V geometry:margin=1in \
  -o report.pdf

# baixa o PDF
from google.colab import files
files.download('report.pdf')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>